In [14]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["HUGGINGFACE_API_KEY"] = os.getenv("HUGGINGFACE_API_KEY")

In [15]:
from langchain_groq import ChatGroq
llm = ChatGroq(model_name="openai/gpt-oss-120b")

In [16]:
from langchain_core.tools import tool

In [17]:
from langchain_tavily import TavilySearch

In [18]:
@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers together."""
    return a * b

In [23]:
multiply.invoke({"a":2, "b": 4})

8

In [26]:
@tool
def search(query: str):
    """Search the web for a query."""
    tavily = TavilySearch()
    result = tavily.invoke(query)
    return result

In [30]:
tools = [search, multiply]

In [32]:
llm_with_tools = llm.bind_tools(tools)

In [33]:
tool_mapping = {tool.name:tool for tool in tools}

In [34]:
tool_mapping

{'search': StructuredTool(name='search', description='Search the web for a query.', args_schema=<class 'langchain_core.utils.pydantic.search'>, func=<function search at 0x00000251E146CA40>),
 'multiply': StructuredTool(name='multiply', description='Multiply two numbers together.', args_schema=<class 'langchain_core.utils.pydantic.multiply'>, func=<function multiply at 0x00000251E1346DE0>)}

In [35]:
tool_mapping["search"]

StructuredTool(name='search', description='Search the web for a query.', args_schema=<class 'langchain_core.utils.pydantic.search'>, func=<function search at 0x00000251E146CA40>)

In [37]:
tool_mapping["multiply"].invoke({"a":2, "b":3})

6